In [1]:
from pycromanager import *
import matplotlib.pyplot as plt
import math
import numpy as np
from skimage import filters, measure, io
import os
from sklearn import svm
from scipy.optimize import curve_fit, minimize_scalar
import time
from IPython.display import clear_output

In [2]:

def compute_edges(proc):
    mean_intensity = np.mean(proc)
    edge_map = filters.sobel(proc)
    mean_edge = np.mean(edge_map)
    return (mean_edge / mean_intensity)

def compute_sharp_edges(proc):
    mean_intensity = np.mean(proc)
    sharpened = filters.unsharp_mask(proc, radius=1.0, amount=1.0)
    edge_map = filters.sobel(sharpened)
    mean_edge = np.mean(edge_map)
    return (mean_edge / mean_intensity)

def compute_mean(proc):
    return (np.mean(proc))

def compute_normalized_std_dev(proc):
    stdd = np.std(proc)
    mean = np.mean(proc)
    return(stdd/mean)

def compute_normalized_variance(proc):
    stdd = np.std(proc)
    mean = np.mean(proc)
    return ((stdd ** 2) / mean)

def compute_redondo(proc):
    w, h = proc.shape
    sum = 0.0
    for i in range(1, w - 2):
        for j in range(1, h - 2):
            p = proc[i - 1, j]
            p += proc[i + 1, j]
            p += proc[i, j - 1]
            p += proc[i, j + 1]
            p -= (4 * proc[i-1, j])
            sum += (p * p)
    return (sum)

def compute_volath(proc):
    w, h = proc.shape
    normsum = np.float64(0.)
    for i in range(1, w - 1):
        for j in range(1, h-1):
            normsum += (proc[i, j]/256 * proc[i + 1, j]/256) - (proc[i-1, j]/256 * proc[i, j]/256)
    return (normsum)

def compute_volath5(proc):
    w, h = proc.shape
    sum = np.float64(0.)
    for i in range(w - 1):
        for j in range(h - 1):
            sum += proc[i,j]/256 * proc[i,j]/256
    mean = np.mean(proc)
    sum -= ((w - 1) * h * mean * mean)
    return (sum)

def compute_tenengrad(proc):
    sum = 0.0
    proc1 = filters.median(proc)
    sobel = filters.sobel(proc1)
    sum = np.sum(sobel)
    return (sum)

def compute_scores(img):
    edgescore = compute_edges(img)
    sharpscore = compute_sharp_edges(img)
    meanscore = compute_mean(img)
    stddevscore = compute_normalized_std_dev(img)
    nvarscore = compute_normalized_variance(img)
    redondoscore = compute_redondo(img)
    #volathscore = compute_volath(img)
    #volath5score = compute_volath5(img)
    tenengradscore = compute_tenengrad(img)
    scores = {'Edge Score':edgescore, 'Sharp Edge Score':sharpscore, 'Mean Score':meanscore, 'Normalized Standard Deviation Score':stddevscore, 'Normalized Variance Score':nvarscore, 
              'Redondo Score':redondoscore, 'Tenengrad Score':tenengradscore} #'Volath Score':volathscore, 'Volath5 Score':volath5score, 
    return(scores)

def Gauss(x, A, B):
    return(A*np.exp(-1*B*x**2))

def compute_fft_bandpass(image, fft_lower_cutoff, fft_upper_cutoff):
    # Get height and width of the image
    height, width = image.shape[:2]

    # Compute the FFT of the image
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    
    # Create a mask first, center square is 1, remaining all zeros
    mask = np.zeros((height, width))
    midpoint = height // 2
    scaled_lower = int(np.round(fft_lower_cutoff / 100 * midpoint))
    start_lower = int(np.round(midpoint - scaled_lower))
    scaled_upper = int(np.round(fft_upper_cutoff / 100 * midpoint))
    start_upper = int(np.round(midpoint - scaled_upper))

    # Apply the circular masks
    x, y = np.ogrid[:height, :width]
    mask_outer = ((y - midpoint)**2 + (x - midpoint)**2 >= scaled_upper**2)
    mask_inner = ((y - midpoint)**2 + (x - midpoint)**2 <= scaled_lower**2)
    mask = np.logical_or(mask_outer, mask_inner)
    
    # Apply mask and inverse DFT
    fshift_masked = fshift * mask
    f_ishift = np.fft.ifftshift(fshift_masked)
    img_back = np.fft.ifft2(f_ishift)
    
    # Take absolute value and calculate the mean
    img_back = np.abs(img_back)
    return np.mean(img_back)

In [3]:
core = Core()
studio = Studio()

In [ ]:
dir = 'C:/Users/thelab/Desktop/TestStack'
nam = 'test'
zdistance = 4
step = 0.1
roi = 0.6

In [ ]:
zdevice = core.get_focus_device()
zpos = core.get_position(zdevice)
start = zpos - zdistance/2
end = zpos + zdistance/2
zs = np.arange(start,end,step)
with Acquisition(directory=dir, name=nam) as acq:
    events = multi_d_acquisition_events(z_start=start, z_end=end, z_step=step)
    acq.acquire(events)
imgs = acq.get_dataset()

In [ ]:
scoreset = []
for zd in range(len(zs)):
    print(str(zd+1)+"/"+str(len(zs)))
    img = imgs.read_image(z=zd)
    h, w = img.shape
    hclip = int(h * (1-roi) / 2)
    wclip = int(w * (1-roi) / 2)
    cropped_img = img[hclip:(h-hclip) , wclip:(w-wclip)]
    scores = compute_scores(cropped_img)
    scoreset.append(scores)

In [ ]:
all_params = {}

keynames = list(scoreset[0].keys())
colscores = [list(x.values()) for x in scoreset]
rowscores = np.swapaxes(np.array(colscores),0,1)
nrowscores = []
#print(rowscores)
for rsco in rowscores:
    rsco = abs(rsco)
    maxi = max(rsco)
    mini = min(rsco)
    nrow = [((x-mini)/(maxi-mini)) for x in rsco]
    nrowscores.append(nrow)
nrowscores = np.array(nrowscores)
#nrowscores = np.array([list(abs(x)/(max(abs(x)))) for x in rowscores])
fig, ax = plt.subplots()
keynames = list(scoreset[0].keys())
xes = list(range(len(nrowscores[0])))
zeroloc = 22
zvals = (np.array(xes)-zeroloc)/(1/step)
smallzsteps = np.arange(zvals[0],zvals[-1],0.01)
for i in range(len(nrowscores)):
    ax.plot(zvals,nrowscores[i], 'o', label=keynames[i])
collist = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i in range(len(nrowscores)):
    parameters, covariance = curve_fit(Gauss, zvals, nrowscores[i])
    all_params[keynames[i]]=[parameters,covariance]
    fit_y = Gauss(smallzsteps,  parameters[0], parameters[1])
    ax.plot(smallzsteps,fit_y, color=collist[i])
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
fig.show()

In [ ]:
fig, axes = plt.subplots(nrows=len(rowscores),ncols=1,sharex=True, figsize=(5,40))
for i in range(len(rowscores)):
    axes[i].set_title(keynames[i])
    axes[i].plot(zvals,rowscores[i], 'o')
    axes[i].set_xlabel('microns')
plt.show


In [ ]:
for k in all_params:
    print(k)
    v = all_params[k]
    print(np.diag(v[1]))

In [ ]:
x = colscores
y = abs(zvals) < 0.25
learner = svm.SVC()
learner.fit(x,y)
#print(learner.predict(x))

In [ ]:
core.snap_image()
tagged_image = core.get_tagged_image()
testim = np.reshape(tagged_image.pix,newshape=[tagged_image.tags['Height'], tagged_image.tags['Width']])
scores = list(compute_scores(testim).values())
print(learner.predict([scores]))

In [ ]:
global dir
global zpositions
global zdevice
global starttime
dir = 'C:/Users/thelab/Desktop/TestStack'
nam = 'test'
numsites = 2
numtimepoints = 2

In [ ]:
core = Core()
studio = Studio()

In [ ]:
def waitEnter(evt):
    input("Press Enter to continue...")
    return(evt)

def waitEnterResetZ(evt):
    input("Press Enter to continue...")
    zpositions[i] = core.get_position(zdevice)
    return(evt)

def saveImg():
    stime = time.perf_counter() - starttime
    tagged_image = core.get_tagged_image()
    testim = np.reshape(tagged_image.pix,newshape=[tagged_image.tags['Height'], tagged_image.tags['Width']])
    io.imsave(os.path.join(os.path.join(dir,str(i)),stime),testim)

def snapAndCompute(zpos):
    zdevice = core.get_focus_device()
    core.set_position(zdevice,zpos)
    print(zpos)
    core.snap_image()
    tagged_image = core.get_tagged_image()
    testim = np.reshape(tagged_image.pix,newshape=[tagged_image.tags['Height'], tagged_image.tags['Width']])
    score = compute_redondo(testim)
    return(score)

def findGenZ(evt):
    zdevice = core.get_focus_device()
    zpos = core.get_position(zdevice)
    #res = minimize_scalar(snapAndCompute, bounds=(-0.5,0.5))
    core.set_position(zdevice,zpos)
    scores = []
    nit = 0
    scores.append([nit,snapAndCompute()])
    offset = 0.5
    while True:
        for nit in [offset*-1,offset]:
            core.set_position(zdevice,nit)
            scores.append([nit,snapAndCompute()])
        offset += 1
        sline = scores.sort()
        if offset > 10:
            return(scores)
    for zm in range(-10,5,1):
        core.set_position(zdevice,(zpos+zm))
        scores.append(snapAndCompute)
    print(scores)
    return(evt)

In [17]:
zdevice = core.get_focus_device()
zpos = core.get_position(zdevice)
print(zpos)
res = minimize_scalar(snapAndCompute, bounds=(zpos-5,zpos+5))
print(res)
core.set_position(zdevice,res.fun)

75.97500000000001


KeyboardInterrupt: 

In [ ]:
zdevice = core.get_focus_device()

posfolder = os.path.join(dir,'ordered')
if not os.path.exists(posfolder):
    os.makedirs(posfolder)
    for i in range(numsites):
        os.makedirs(os.path.join(posfolder,str(i)))

xyzpositions = []
zpositions = []
xypositions = []
for site in range(numsites):
    sitefolder = os.path.join(posfolder,str(i))
    waitEnter('evt')
    #core.snap_image()
    #tagged_image = core.get_tagged_image()
    zpos = core.get_position(zdevice)
    xpos = core.get_x_position()
    ypos = core.get_y_position()
    xyzpositions.append([xpos,ypos,zpos])
    xypositions.append([xpos,ypos])
    zpositions.append([zpos])
    #pixels = np.reshape(tagged_image.pix,newshape=[tagged_image.tags['Height'], tagged_image.tags['Width']])
    #io.imsave(os.path.join(sitefolder,'before.tif'),pixels)

while True:
    go = input("Before images done, enter 'E' to start acquisition")
    if go == 'E':
        break

In [ ]:
with Acquisition(directory=dir, name=nam, post_hardware_hook_fn=findGenZ) as acq:
    events = multi_d_acquisition_events(num_time_points=numtimepoints,time_interval_s=0.1,xyz_positions=xyzpositions)
    acq.acquire(events)

#with Acquisition(directory=dir, name=nam, post_hardware_hook_fn=findGenZ) as acq:
#    events = multi_d_acquisition_events(num_time_points=numtimepoints,xyz_positions=xyzpositions)
#    acq.acquire(events)

In [ ]:
def snap_image():
    # acquire an image and display it
    core.snap_image()
    tagged_image = core.get_tagged_image()
    # get the pixels in numpy array and reshape it according to its height and width
    image_array = np.reshape(
        tagged_image.pix,
        newshape=[-1, tagged_image.tags["Height"], tagged_image.tags["Width"]],
    )
    # for display, we can scale the image into the range of 0~255
    image_array = (image_array / image_array.max() * 255).astype("uint8")
    # return the first channel if multiple exists
    return image_array[0, :, :]


for i in range(60):
    clear_output(wait=True)
    plt.figure()
    plt.title(i)
    plt.imshow(snap_image())
    plt.show()


In [ ]:
acq = MagellanAcquisition(magellan_explore=True)
acq.await_completion()